In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

import pprint

## 1. Define model and parameters

In [2]:
wandb.login()

# Define parameters 

num_epochs = 1000
project_name = 'finetuning_model_architecture'
train_ratio = 0.8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


## 2. Load data

In [3]:
# Load the list of dictionaries
data_dict_list = torch.load('../data/dataset_1pm_0-1382.pt')

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]

# # Apply normalization to your dataset
dataset_normalized = gio.normalize_dataset(datalist)

baseline_error = gio.compute_baseline_error(datalist)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.005565311759710312


## 3. Split into train and test set

In [4]:
train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=20, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=20, train_ratio=train_ratio)
# n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

Total dataset length: 1382
Training subset length: 1100
Total dataset length: 1382
Validation subset length: 260


## 4. Train the model

We first find a good model for one batch. 

In [5]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None, device=None, early_stopping=None):
    for epoch in range(config.epochs):
        model.train()
        for idx, data in enumerate(train_dl):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()
    return val_loss, epoch

In [6]:
# Currently best architecture ! 

batch_size = 16
output_layer_parameter = 'gat'
hidden_size_parameter = 16
gat_layer_parameter = 0
gcn_layer_parameter = 0
lr = 0.001

wandb.login()

train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)

wandb.init(
    project=project_name,
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "early_stopping_patience": 10,
        "hidden_layer_size": hidden_size_parameter,
        "gat_layers": gat_layer_parameter,
        "gcn_layers": gcn_layer_parameter,
        "output_layer": output_layer_parameter,
        # "dropout": 0.15,
    }
)
config = wandb.config

print("output_layer: ", output_layer_parameter)
print("hidden_size: ", hidden_size_parameter)
print("gat_layers: ", gat_layer_parameter)
print("gcn_layers: ", gcn_layer_parameter)

gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
model = gnn_instance.to(device)

best_val_loss, best_epoch = train(model, config=config, 
                                loss_fct=torch.nn.MSELoss(), 
                                optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)

Total dataset length: 1382
Training subset length: 1104
Total dataset length: 1382
Validation subset length: 272


output_layer:  gcn
hidden_size:  16
gat_layers:  0
gcn_layers:  0
Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GCNConv(16, 1)
)
epoch: 0, validation loss: 0.005556610741597765
epoch: 1, validation loss: 0.005541034864590448
epoch: 2, validation loss: 0.0054847901577458665
epoch: 3, validation loss: 0.005454170572407106
epoch: 4, validation loss: 0.005423781051136115
epoch: 5, validation loss: 0.00539033191607279
epoch: 6, validation loss: 0.005340714747195735
epoch: 7, validation loss: 0.005216823123833712
epoch: 8, validation loss: 0.005054894889540532
epoch: 9, valid

wandb: Network error (ConnectionError), entering retry loop.


epoch: 51, validation loss: 0.004200166611767867
EarlyStopping counter: 1 out of 10
epoch: 52, validation loss: 0.004202314953812782
EarlyStopping counter: 2 out of 10
epoch: 53, validation loss: 0.004181406707228983
epoch: 54, validation loss: 0.004172874368069803
epoch: 55, validation loss: 0.004171533838791006
epoch: 56, validation loss: 0.004164169542491436
epoch: 57, validation loss: 0.004170472158447784
EarlyStopping counter: 1 out of 10
epoch: 58, validation loss: 0.004160016528604662
epoch: 59, validation loss: 0.004162019255625851
EarlyStopping counter: 1 out of 10
epoch: 60, validation loss: 0.004197090508087593
EarlyStopping counter: 2 out of 10
epoch: 61, validation loss: 0.004156437346383053
epoch: 62, validation loss: 0.004150300615412348
epoch: 63, validation loss: 0.004149044946055202
epoch: 64, validation loss: 0.004148545209318399
epoch: 65, validation loss: 0.004146153682514149
epoch: 66, validation loss: 0.00414287655011696
epoch: 67, validation loss: 0.004143792560

KeyboardInterrupt: 

In [ ]:
import pickle

# Save the state dictionary and configuration
state_dict = model.state_dict()
config = {
    'in_channels': 3,
    'out_channels': 1,
    'hidden_size': hidden_size_parameter,
    'gat_layers': gat_layer_parameter,
    'gcn_layers': gcn_layer_parameter,
    'output_layer': output_layer_parameter,
    'other_config': config  # Include other configuration parameters as needed
}

# Save to a pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump({'state_dict': state_dict, 'config': config}, f)